<a href="https://colab.research.google.com/github/sergejhorvat/TensorFlow-Data-and-Deployment-Specialization/blob/master/Data%20Pipelines%20with%20TensorFlow%20Data%20Services/Week%202/Examples/csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.


In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Load CSV data

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/csv"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial provides an example of how to load CSV data from a file into a `tf.data.Dataset`.

The data used in this tutorial are taken from the Titanic passenger list. The model will predict the likelihood a passenger survived based on characteristics like age, gender, ticket class, and whether the person was traveling alone.

## Setup

In [0]:
import functools

import numpy as np
import tensorflow as tf

In [0]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [0]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

## Load data

To start, let's look at the top of the CSV file to see how it is formatted.

In [0]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


You can [load this using pandas](pandas_dataframe.ipynb), and pass the NumPy arrays to TensorFlow. If you need to scale up to a large set of files, or need a loader that integrates with [TensorFlow and tf.data](../../guide/data.ipynb) then use the `tf.data.experimental.make_csv_dataset` function:

The only column you need to identify explicitly is the one with the value that the model is intended to predict. 

In [0]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

Now read the CSV data from the file and create a dataset. 

(For the full documentation, see `tf.data.experimental.make_csv_dataset`)


In [0]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [0]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

Each item in the dataset is a batch, represented as a tuple of (*many examples*, *many labels*). The data from the examples is organized in column-based tensors (rather than row-based tensors), each with as many elements as the batch size (5 in this case).

It might help to see this yourself.

In [0]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'male']
age                 : [34. 40. 48. 28. 48.]
n_siblings_spouses  : [0 0 1 0 1]
parch               : [0 0 2 0 0]
fare                : [ 8.05  31.    65.     7.225 76.729]
class               : [b'Third' b'First' b'Second' b'Third' b'First']
deck                : [b'unknown' b'A' b'unknown' b'unknown' b'D']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Cherbourg' b'Cherbourg']
alone               : [b'y' b'y' b'n' b'y' b'n']


As you can see, the columns in the CSV are named. The dataset constructor will pick these names up automatically. If the file you are working with does not contain the column names in the first line, pass them in a list of strings to  the `column_names` argument in the `make_csv_dataset` function.

In [0]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'male' b'male' b'female' b'female']
age                 : [28. 60. 28. 45. 28.]
n_siblings_spouses  : [8 1 0 1 1]
parch               : [2 1 0 1 0]
fare                : [ 69.55   39.      7.896 164.867  14.454]
class               : [b'Third' b'Second' b'Third' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'n' b'n' b'y' b'n' b'n']


This example is going to use all the available columns. If you need to omit some columns from the dataset, create a list of just the columns you plan to use, and pass it into the (optional) `select_columns` argument of the constructor.


In [0]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [ 2. 30. 32. 32. 26.]
n_siblings_spouses  : [3 0 0 0 0]
class               : [b'Third' b'Second' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
alone               : [b'n' b'y' b'y' b'y' b'y']


## Data preprocessing

A CSV file can contain a variety of data types. Typically you want to convert from those mixed types to a fixed length vector before feeding the data into your model.

TensorFlow has a built-in system for describing common input conversions: `tf.feature_column`, see [this tutorial](../keras/feature_columns) for details.


You can preprocess your data using any tool you like (like [nltk](https://www.nltk.org/) or [sklearn](https://scikit-learn.org/stable/)), and just pass the processed output to TensorFlow. 


The primary advantage of doing the preprocessing inside your model is that when you export the model it includes the preprocessing. This way you can pass the raw data directly to your model.

### Continuous data

If your data is already in an appropriate numeric format, you can pack the data into a vector before passing it off to the model:

In [0]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [40.5  7.  20.  28.  28. ]
n_siblings_spouses  : [0. 0. 0. 0. 1.]
parch               : [2. 2. 0. 0. 0.]
fare                : [14.5  26.25  7.05  0.   24.  ]


In [0]:
example_batch, labels_batch = next(iter(temp_dataset)) 

Here's a simple function that will pack together all the columns:

In [0]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

Apply this to each element of the dataset:

In [0]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[28.     0.     0.    13.   ]
 [ 7.     4.     1.    39.688]
 [33.     0.     2.    26.   ]
 [ 9.     3.     2.    27.9  ]
 [ 2.     3.     1.    21.075]]

[1 0 1 0 0]


If you have mixed datatypes you may want to separate out these simple-numeric fields. The `tf.feature_column` api can handle them, but this incurs some overhead and should be avoided unless really necessary. Switch back to the mixed dataset:

In [0]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'female']
age                 : [48. 22. 25.  9. 18.]
n_siblings_spouses  : [0 0 1 1 0]
parch               : [0 1 0 1 1]
fare                : [ 7.854 55.    26.    15.9   23.   ]
class               : [b'Third' b'First' b'Second' b'Third' b'Second']
deck                : [b'unknown' b'E' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'n' b'n' b'n' b'n']


In [0]:
example_batch, labels_batch = next(iter(temp_dataset)) 

So define a more general preprocessor that selects a list of numeric features and packs them into a single column:

In [0]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [0]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [0]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'male' b'female' b'male']
class               : [b'First' b'First' b'Third' b'Second' b'Third']
deck                : [b'unknown' b'E' b'unknown' b'unknown' b'F']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'n' b'y' b'n' b'y']
numeric             : [[28.     0.     0.    31.   ]
 [50.     1.     0.    55.9  ]
 [33.     0.     0.     8.654]
 [41.     0.     1.    19.5  ]
 [19.     0.     0.     7.65 ]]


In [0]:
example_batch, labels_batch = next(iter(packed_train_data)) 

#### Data Normalization

Continuous data should always be normalized.

In [0]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [0]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std


Now create a numeric column. The `tf.feature_columns.numeric_column` API accepts a `normalizer_fn` argument, which will be run on each batch.

Bind the `MEAN` and `STD` to the normalizer fn using [`functools.partial`](https://docs.python.org/3/library/functools.html#functools.partial).

In [0]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7fa42155ca60>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

When you train the model, include this feature column to select and center this block of numeric data:

In [0]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[30.   ,  0.   ,  0.   , 12.475],
       [19.   ,  0.   ,  0.   ,  8.158],
       [46.   ,  1.   ,  0.   , 61.175],
       [31.   ,  0.   ,  0.   , 10.5  ],
       [28.   ,  0.   ,  0.   , 52.   ]], dtype=float32)>

In [0]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 0.029, -0.474, -0.479, -0.401],
       [-0.85 , -0.474, -0.479, -0.48 ],
       [ 1.308,  0.395, -0.479,  0.491],
       [ 0.109, -0.474, -0.479, -0.437],
       [-0.13 , -0.474, -0.479,  0.323]], dtype=float32)

The mean based normalization used here requires knowing the means of each column ahead of time.

### Categorical data

Some of the columns in the CSV data are categorical columns. That is, the content should be one of a limited set of options.

Use the `tf.feature_column` API to create a collection with a `tf.feature_column.indicator_column` for each categorical column.


In [0]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [0]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [0]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


This will be become part of a data processing input later when you build the model.

### Combined preprocessing layer

Add the two feature column collections and pass them to a `tf.keras.layers.DenseFeatures` to create an input layer that will extract and preprocess both input types:

In [0]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [0]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.029 -0.474
 -0.479 -0.401  0.     1.   ]


## Build the model

Build a `tf.keras.Sequential`, starting with the `preprocessing_layer`.

In [0]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

## Train, evaluate, and predict

Now the model can be instantiated and trained.

In [0]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [0]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 0s 2ms/step - loss: 0.5049 - accuracy: 0.7368
Epoch 2/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4285 - accuracy: 0.8118
Epoch 3/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4072 - accuracy: 0.8230
Epoch 4/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3959 - accuracy: 0.8341
Epoch 5/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3845 - accuracy: 0.8341
Epoch 6/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3728 - accuracy: 0.8389
Epoch 7/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3671 - accuracy: 0.8469
Epoch 8/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3606 - accuracy: 0.8517
Epoch 9/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3540 - accuracy: 0.8533
Epoch 10/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3472 - accuracy: 0.8437

Once the model is trained, you can check its accuracy on the `test_data` set.

In [0]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 0s 1ms/step - loss: 0.4453 - accuracy: 0.8523


Test Loss 0.44534531235694885, Test Accuracy 0.8522727489471436


Use `tf.keras.Model.predict` to infer labels on a batch or a dataset of batches.

In [0]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))


Predicted survival: 1.06%  | Actual outcome:  DIED
Predicted survival: 13.51%  | Actual outcome:  SURVIVED
Predicted survival: 19.02%  | Actual outcome:  SURVIVED
Predicted survival: 14.62%  | Actual outcome:  SURVIVED
Predicted survival: 14.38%  | Actual outcome:  SURVIVED
